<a href="https://colab.research.google.com/github/Radyko/MLQuantStrategy/blob/main/AlgorithminTradingUnsupervisedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=d88fa4948923126ee8b497f49426f4e5981e784ac0def7f288835784941dd512
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [117]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-08-01'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df


[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-03 A        44.222782   47.090000   47.180000   46.900002   47.099998   
           AAL      32.271362   33.480000   33.959999   33.169998   33.470001   
           AAPL     24.263590   26.447500   26.459999   26.192499   26.202499   
           ABBV     47.158066   66.570000   67.000000   66.220001   66.809998   
           ABT      38.829514   44.950001   45.090000   44.830002   45.070000   
...                       ...         ...         ...         ...         ...   
2024-07-31 XYL     133.500000  133.500000  135.429993  132.600006  134.880005   
           YUM     132.830002  132.830002  134.029999  131.610001  132.759995   
           ZBH     111.349998  111.349998  113.080002  110.120003  111.510002   
           ZBRA    351.190002  351.190002  359.690002  348.989990  357.779999   
           ZTS     180.039993  180.039993  183.259995  179.830002  182.500000   

Price                   volume  
date       ticker               
2016-08-03 A         1508900.0  
           AAL       8254200.0  
           AAPL    120810400.0  
           ABBV      6355100.0  
           ABT       7783100.0  
...                        ...  
2024-07-31 XYL       2439500.0  
           YUM       1943100.0  
           ZBH       1592800.0  
           ZBRA       634400.0  
           ZTS       1740100.0  

[990317 rows x 6 columns]

# 2. Calculate features and technical indicators for each stock.


*   Garman-Klass Volatility
*   RSI
*   Bollinger Bands
*   ATR
*   MACD
*   Dollar Volume

Garman Klass Volatility is an approximation used to measure intraday volatility of a given asset.

Garman-Klass Volatility = (((ln(High) - ln(Low))^2)/2) -(2ln(2)-1)(ln(Adj Close) - ln(Open))^2.


In [118]:
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['adj close']) - np.log(df['open']))**2)
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length = 20))
pandas_ta.bbands(close=df.xs('AAPL', level=1)['adj close'], length = 20)

def calculate_bbands(x):
    bbands_result = pandas_ta.bbands(close=np.log1p(pd.Series(x)), length = 20)
    return bbands_result if bbands_result is not None else pd.DataFrame()  # Return an empty DataFrame if bbands_result is None
    #function needed, as iloc cannot work if input is None.

df['bb_lower'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:,0] if not calculate_bbands(x).empty else None)

df['bb_middle'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:,1] if not calculate_bbands(x).empty else None)

df['bb_upper'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:,2] if not calculate_bbands(x).empty else None)

def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close = stock_data['close'], length = 14)
  return atr.sub(atr.mean()).div(atr.std())


df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

def compute_macd(close):
  macd_result = pandas_ta.macd(close=close, length = 20)
  if macd_result is not None:  # Check if macd_result is not None
    macd = macd_result.iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
  else:
    return None  # Return None if macd_result is None

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close'] * df['volume'])/1e6
#dollar volume in millions

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-03 A        44.222782   47.090000   47.180000   46.900002   47.099998   
           AAL      32.271362   33.480000   33.959999   33.169998   33.470001   
           AAPL     24.263590   26.447500   26.459999   26.192499   26.202499   
           ABBV     47.158066   66.570000   67.000000   66.220001   66.809998   
           ABT      38.829514   44.950001   45.090000   44.830002   45.070000   
...                       ...         ...         ...         ...         ...   
2024-07-31 XYL     133.500000  133.500000  135.429993  132.600006  134.880005   
           YUM     132.830002  132.830002  134.029999  131.610001  132.759995   
           ZBH     111.349998  111.349998  113.080002  110.120003  111.510002   
           ZBRA    351.190002  351.190002  359.690002  348.989990  357.779999   
           ZTS     180.039993  180.039993  183.259995  179.830002  182.500000   

Price                   volume  garman_klass_vol        rsi  bb_lower  \
date       ticker                                                       
2016-08-03 A         1508900.0         -0.001517        NaN       NaN   
           AAL       8254200.0         -0.000237        NaN       NaN   
           AAPL    120810400.0         -0.002231        NaN       NaN   
           ABBV      6355100.0         -0.046807        NaN       NaN   
           ABT       7783100.0         -0.008564        NaN       NaN   
...                        ...               ...        ...       ...   
2024-07-31 XYL       2439500.0          0.000182  44.378010  4.888463   
           YUM       1943100.0          0.000166  53.203270  4.838324   
           ZBH       1592800.0          0.000351  52.383683  4.666365   
           ZBRA       634400.0          0.000322  67.599872  5.729809   
           ZTS       1740100.0          0.000107  56.187626  5.159026   

Price              bb_middle  bb_upper       atr      macd  dollar_volume  
date       ticker                                                          
2016-08-03 A             NaN       NaN       NaN       NaN      66.727756  
           AAL           NaN       NaN       NaN       NaN     266.374279  
           AAPL          NaN       NaN       NaN       NaN    2931.293996  
           ABBV          NaN       NaN       NaN       NaN     299.694224  
           ABT           NaN       NaN       NaN       NaN     302.213987  
...                      ...       ...       ...       ...            ...  
2024-07-31 XYL      4.928355  4.968247  1.892789 -0.278173     325.673250  
           YUM      4.866018  4.893712  0.721489 -0.686611     258.101977  
           ZBH      4.702034  4.737703 -0.568448  0.190412     177.358278  
           ZBRA     5.790869  5.851929  0.521713  0.873963     222.794938  
           ZTS      5.190371  5.221715  0.716857  0.688203     313.287592  

[990317 rows x 14 columns]

#3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.



*  To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.


*  Calculate 5-year rolling average of dollar volume for each stock before filtering.








In [119]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

data['dollar_volume'] = (data.loc[:,'dollar_volume'].unstack('ticker').rolling(5*12, min_periods = 12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending = False))

data = data[data['dollar_vol_rank'] < 150].drop(['dollar_vol_rank','dollar_volume'], axis = 1)

data

adj close  garman_klass_vol        rsi  bb_lower  \
date       ticker                                                      
2017-08-31 AAL      43.511292         -0.000146  37.643915  3.741600   
           AAPL     38.485268         -0.001422  65.282991  3.621129   
           ABBV     55.412022         -0.032460  68.425708  3.944742   
           ABT      45.090454         -0.004445  62.800448  3.773206   
           ACN     117.201653         -0.004114  59.510444  4.746614   
...                       ...               ...        ...       ...   
2024-07-31 VZ       40.520000          0.000144  51.735733  3.687073   
           WFC      58.890114          0.000024  50.270815  4.063394   
           WMT      68.639999          0.000074  51.666269  4.241554   
           XOM     118.589996          0.000021  57.851744  4.716281   
           ZTS     180.039993          0.000107  56.187626  5.159026   

                   bb_middle  bb_upper       atr      macd  
date       ticker                                           
2017-08-31 AAL      3.839023  3.936447  0.379254 -1.850155  
           AAPL     3.649403  3.677678 -1.152032 -0.011288  
           ABBV     3.983809  4.022875 -1.528612  0.061339  
           ABT      3.799478  3.825750 -1.352457  0.070556  
           ACN      4.759914  4.773213 -1.340890 -0.058480  
...                      ...       ...       ...       ...  
2024-07-31 VZ       3.728617  3.770161 -0.191153 -0.152062  
           WFC      4.095216  4.127037  0.633665  0.131169  
           WMT      4.261496  4.281437  0.787756  0.648607  
           XOM      4.755448  4.794615  0.597315  0.679436  
           ZTS      5.190371  5.221715  0.716857  0.688203  

[12516 rows x 8 columns]

# 4. Calculate Monthly Returns for different time horizons as features

*   To capture time series dynamics that reflect momentum patterns, we compute historical returns using the method .pct_change(lag), which shows returns over various montlhy periods as identified by lags.



In [120]:
def calculate_returns(df):

  outlier_cutoff = 0.005 #arbitrary

  lags = [1,2,3,6,9,12]

  for lag in lags:
    df[f'return_{lag}m'] = (df['adj close']
                          .pct_change(lag)
                          .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                          .add(1)
                          .pow(1/lag)
                          .sub(1))

  return df


data = data.groupby(level = 1, group_keys= False).apply(calculate_returns).dropna()

data

adj close  garman_klass_vol        rsi  bb_lower  \
date       ticker                                                      
2018-08-31 AAL      39.726204         -0.000024  55.978002  3.603652   
           AAPL     54.224766         -0.000647  81.062422  3.896835   
           ABBV     73.114151         -0.029147  49.722382  4.287913   
           ABT      60.295719         -0.004198  64.178195  4.055374   
           ACN     154.357224         -0.003021  63.975164  4.977965   
...                       ...               ...        ...       ...   
2024-07-31 VRTX    495.720001          0.000114  62.146045  6.165930   
           VZ       40.520000          0.000144  51.735733  3.687073   
           WFC      58.890114          0.000024  50.270815  4.063394   
           WMT      68.639999          0.000074  51.666269  4.241554   
           XOM     118.589996          0.000021  57.851744  4.716281   

                   bb_middle  bb_upper       atr      macd  return_1m  \
date       ticker                                                       
2018-08-31 AAL      3.665384  3.727116  0.003666  0.718917   0.026445   
           AAPL     3.952077  4.007320 -0.998122  0.434840   0.200423   
           ABBV     4.314163  4.340413 -0.645670  0.023127   0.040659   
           ABT      4.091499  4.127625 -1.228504  0.428157   0.019835   
           ACN      5.015797  5.053629 -1.118371  0.181193   0.061131   
...                      ...       ...       ...       ...        ...   
2024-07-31 VRTX     6.195736  6.225542  1.373761  1.324232   0.057604   
           VZ       3.728617  3.770161 -0.191153 -0.152062  -0.001379   
           WFC      4.095216  4.127037  0.633665  0.131169  -0.000842   
           WMT      4.261496  4.281437  0.787756  0.648607   0.013735   
           XOM      4.755448  4.794615  0.597315  0.679436   0.030142   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2018-08-31 AAL      0.034006  -0.023150  -0.046843  -0.023509   -0.007555  
           AAPL     0.110862   0.069228   0.043001   0.033036    0.028984  
           ABBV     0.022996  -0.006715  -0.027533   0.001987    0.023371  
           ABT      0.049211   0.029514   0.018801   0.020703    0.024511  
           ACN      0.016611   0.027753   0.009652   0.015882    0.023213  
...                      ...        ...        ...        ...         ...  
2024-07-31 VRTX     0.043399   0.080649   0.022652   0.035512    0.028860  
           VZ       0.000402   0.014092  -0.001977   0.021535    0.020476  
           WFC     -0.004852   0.002025   0.030523   0.047980    0.023656  
           WMT      0.021663   0.050878   0.038538   0.027229    0.022525  
           XOM      0.005655   0.003604   0.027048   0.015688    0.011331  

[10369 rows x 14 columns]

# 5. Download Fama-French Factors and Calculate Rolling Factor Betas



*   We will introduce the Fama-French data to estimate the exposure of assets to common risk factors using linear regression.
*   The five Fama-French factors, namely market risk, size, value, operating profitability, and investment have been shown to imperically explain asset returns in the past and are commonly used to asses the risk/return profile of portfolios.
*   We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.





In [121]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start = '2010')[0].drop('RF', axis =1)
factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data



Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-08-31 AAL     0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.026445
           AAPL    0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.200423
           ABBV    0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.040659
           ABT     0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.019835
           ACN     0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.061131
...                   ...     ...     ...     ...     ...        ...
2024-06-30 VRTX    0.0278 -0.0433 -0.0331  0.0049 -0.0182   0.029385
           VZ      0.0278 -0.0433 -0.0331  0.0049 -0.0182   0.002187
           WFC     0.0278 -0.0433 -0.0331  0.0049 -0.0182  -0.008845
           WMT     0.0278 -0.0433 -0.0331  0.0049 -0.0182   0.029653
           XOM     0.0278 -0.0433 -0.0331  0.0049 -0.0182  -0.018250

[10227 rows x 6 columns]

* Filter out stocks with less than 10 months of data.

In [122]:
observations = factor_data.groupby(level = 1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-08-31 AAL     0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.026445
           AAPL    0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.200423
           ABBV    0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.040659
           ABT     0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.019835
           ACN     0.0344  0.0065 -0.0400 -0.0031 -0.0270   0.061131
...                   ...     ...     ...     ...     ...        ...
2024-06-30 VRTX    0.0278 -0.0433 -0.0331  0.0049 -0.0182   0.029385
           VZ      0.0278 -0.0433 -0.0331  0.0049 -0.0182   0.002187
           WFC     0.0278 -0.0433 -0.0331  0.0049 -0.0182  -0.008845
           WMT     0.0278 -0.0433 -0.0331  0.0049 -0.0182   0.029653
           XOM     0.0278 -0.0433 -0.0331  0.0049 -0.0182  -0.018250

[10198 rows x 6 columns]

* Calculate Rolling Factor Betas.

In [123]:
betas = (factor_data.groupby(level=1,
                      group_keys=False)
    .apply(lambda x: RollingOLS(endog=x['return_1m'],
                                exog = sm.add_constant(x.drop('return_1m', axis = 1)),
                                window= min(24, x.shape[0]),
                                min_nobs=len(x.columns)+1)
    .fit(params_only = True)
    .params
    .drop('const', axis = 1)))

betas


Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2018-08-31 AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-06-30 VRTX    0.489479  0.498689 -0.378682  0.472388  0.418508
           VZ      0.646521 -1.234068  0.594020 -0.054807 -0.053215
           WFC     1.021367  0.344371  1.209066 -0.499867 -0.755414
           WMT     0.480750  0.179873 -0.134151  0.935791  0.005896
           XOM     0.742724  0.043762  0.430496  0.839707  0.497997

[10198 rows x 5 columns]

In [129]:
# factors = ['Mkt-RF',	'SMB',	'HML',	'RMW',	'CMA']

# data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys = False)[factors].apply(lambda x: x.fillna(x.mean()))


data = data.drop('adj close', axis = 1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10159 entries, (Timestamp('2018-08-31 00:00:00'), 'AAL') to (Timestamp('2024-07-31 00:00:00'), 'XOM')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   garman_klass_vol  10159 non-null  float64
 1   rsi               10159 non-null  float64
 2   bb_lower          10159 non-null  float64
 3   bb_middle         10159 non-null  float64
 4   bb_upper          10159 non-null  float64
 5   atr               10159 non-null  float64
 6   macd              10159 non-null  float64
 7   return_1m         10159 non-null  float64
 8   return_2m         10159 non-null  float64
 9   return_3m         10159 non-null  float64
 10  return_6m         10159 non-null  float64
 11  return_9m         10159 non-null  float64
 12  return_12m        10159 non-null  float64
 13  Mkt-RF            10159 non-null  float64
 14  SMB               10159 non-null  float64
 15  HML        